### Model registry and Endpoint deployment 

In [11]:
from google.cloud import aiplatform
from google.api_core.exceptions import FailedPrecondition

PROJECT          = "click-through-rate-prediction"
REGION           = "us-central1"
ENDPOINT_DISPLAY = "ctr_xgb_endpoint"   # same as earlier

aiplatform.init(project=PROJECT, location=REGION)

# Find endpoint by display name (created earlier)
endpoint = [e for e in aiplatform.Endpoint.list() if e.display_name == ENDPOINT_DISPLAY][0]

# 1) Undeploy everything currently there (keeps the endpoint)
for dep in endpoint.list_models():
    try:
        print("Undeploying:", dep.id, dep.display_name)
        endpoint.undeploy(deployed_model_id=dep.id)
    except FailedPrecondition as e:
        print("Skip undeploy:", e)

# 2) Deploy the *latest uploaded* model by display name
MODEL_DISPLAY = "ctr_xgb_v2_named"  # same name you used when uploading
# Pick the most recent resource with that display name
models = sorted(
    [m for m in aiplatform.Model.list(filter=f'display_name="{MODEL_DISPLAY}"')],
    key=lambda m: m.create_time,
    reverse=True
)
model = models[0]
print("Deploying model:", model.resource_name)

model.deploy(
    endpoint=endpoint,
    machine_type="n1-standard-2",
    min_replica_count=1,
    max_replica_count=1,
    traffic_percentage=100,
)
print("Deployed. Now serving:", [(m.id, m.display_name) for m in endpoint.list_models()])


Undeploying: 8135566909379706880 ctr_xgb_v2_named
Deploying model: projects/87568676021/locations/us-central1/models/5156510522664812544
Deployed. Now serving: [('8498106679383031808', 'ctr_xgb_v2_named')]


In [16]:
from google.cloud import aiplatform
PROJECT="click-through-rate-prediction"; REGION="us-central1"

aiplatform.init(project=PROJECT, location=REGION)
eps = aiplatform.Endpoint.list()
for e in eps:
    print(e.display_name, "→", e.resource_name)


ctr_xgb_endpoint → projects/87568676021/locations/us-central1/endpoints/2995591942085017600


In [17]:
endpoint = [e for e in aiplatform.Endpoint.list() if e.display_name=="ctr_xgb_endpoint"][0]
print("Deployed models:", [(m.id, m.model, m.display_name) for m in endpoint.list_models()])


Deployed models: [('8498106679383031808', 'projects/87568676021/locations/us-central1/models/5156510522664812544', 'ctr_xgb_v2_named')]
